In [29]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.path.dirname("."), '..'))
sys.path.insert(0, parent_dir)
from bsmodel import BSModel
import numpy as np
import matplotlib.pyplot as plt
import bridgestan as bs
from scipy.stats import norm
from ipywidgets import interact, FloatSlider

In [30]:
homedir = "/mnt/c/Users/jccer/"
bs.set_bridgestan_path(homedir + "bridgestan")

In [31]:
bs_model1 = BSModel(stan_file = homedir + "mcmcexperiments/stan/one_exponential.stan",
                   data_file = homedir + "mcmcexperiments/stan/one_exponential.json")
bs_model2 = BSModel(stan_file = homedir + "mcmcexperiments/stan/one_normal.stan",
                   data_file = homedir + "mcmcexperiments/stan/one_normal.json")
bs_model3 = BSModel(stan_file = homedir + "mcmcexperiments/stan/one_t.stan",
                   data_file = homedir + "mcmcexperiments/stan/one_t.json")

In [33]:
class SinhArcsinhDistribution():
    def __init__(self, m = 0.0, s = 0.0, d = 0.0, e = 0.0):
        self.m = m
        self.s = s
        self.d = d
        self.e = e

    def logpdf(self, x):
        m, s, d, e = self.m, self.s, self.d, self.e
        z = (x - m) / s
        out = -0.5 * np.log(2 * np.pi)
        out += -0.25 * (np.cosh(2 * d * np.arcsinh(z) - 2 * e) - 1)
        out += np.log(np.cosh(d * np.arcsinh(z) - e))
        out += np.log(d) - np.log(s)
        out += -0.5 * np.log(1 + z**2)
        return out

    def pdf(self, x):
        return np.exp(self.logpdf(x))

    def update_parameters(self, m, s, d, e):
        self.m, self.s, self.d, self.e = m, s, d, e

    def plot(self, mu_line = None, sigma_line = None):
        x = np.linspace(self.m - 4 * self.s, self.m + 4 * self.s, 500)
        y = self.pdf(x)

        plt.figure(figsize = (8,5))
        plt.plot(x, y, label = fr"$\log q(x|m,s,d,e)$")

        if mu_line is not None:
            plt.axvline(mu_line, color='red', linestyle='--', linewidth=1.5, label=r"$\mu$")
        if sigma_line is not None:
            plt.axvline(mu_line - sigma_line, color='gray', linestyle=':', linewidth=1.2, label=r"$\mu - \sigma$")
            plt.axvline(mu_line + sigma_line, color='gray', linestyle=':', linewidth=1.2, label=r"$\mu + \sigma$")

        plt.title("SAS Distribution with μ and σ")
        plt.xlabel("x")
        plt.ylabel("PDF")
        plt.grid(True)
        plt.legend()
        plt.show()

SASDistribution = SinhArcsinhDistribution()
interact(lambda m, s, d, e, mu_overlay, sigma_overlay: (
    SASDistribution.update_parameters(m, s, d, e), 
    SASDistribution.plot(mu_line = mu_overlay, sigma_line = sigma_overlay)),
    m = FloatSlider(value = 0, min = -10, max = 10, step = 0.01, description = 'm'),
    s = FloatSlider(value = 1, min = 0.01, max = 10, step = 0.01, description = 's'),
    d = FloatSlider(value = 1, min = 0.01, max = 10, step = 0.01, description = 'd'),
    e = FloatSlider(value = 0, min = -10, max = 10, step = 0.01, description = 'e'),
    mu_overlay = FloatSlider(value = 0, min = -10, max = 10, step = 0.01, description = 'μ overlay'),
    sigma_overlay = FloatSlider(value = 0.01, min = 0.01, max = 10, step = 0.01, description = 'σ overlay'))

interactive(children=(FloatSlider(value=0.0, description='m', max=10.0, min=-10.0, step=0.01), FloatSlider(val…

<function __main__.<lambda>(m, s, d, e, mu_overlay, sigma_overlay)>

In [34]:
xx = np.linspace(-10, 10, 301)
fxx = [np.exp(bs_model1.log_density(np.array([xxn]), propto = False)) for xxn in xx]

def plot_combined(mu = 0.0, sigma = 1.0):
    x = np.linspace(mu - 5 * sigma, mu + 5 * sigma, 1000)
    NormalDistribution = norm(loc = mu, scale = sigma)
    y = NormalDistribution.pdf(x)

    plt.figure(figsize = (8, 5))
    
    plt.plot(x, y, label = f'Normal: μ = {mu:.2f}, σ = {sigma:.2f}', color = 'blue')
    plt.axvline(mu, color = 'red', linestyle = '--', label = 'Mean (μ)')
    plt.plot(xx, fxx, label = 'Bridgestan Model', color = 'green')
    
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.title('Bridgestan Model vs Normal Distribution')
    plt.grid(True)
    plt.minorticks_on()
    plt.legend()
    plt.show()

interact(plot_combined,
         mu = FloatSlider(value = 0.0, min = -10.0, max = 10.0, step = 0.01, description = 'μ (mean)'),
         sigma = FloatSlider(value = 0.01, min = 0.01, max = 10.0, step = 0.01, description = 'σ (std)'))

interactive(children=(FloatSlider(value=0.0, description='μ (mean)', max=10.0, min=-10.0, step=0.01), FloatSli…

<function __main__.plot_combined(mu=0.0, sigma=1.0)>

In [35]:
xx = np.linspace(-10, 10, 301)
fxx = [np.exp(bs_model2.log_density(np.array([xxn]), propto = False)) for xxn in xx]

interact(plot_combined,
         mu = FloatSlider(value = 0.0, min = -10.0, max = 10.0, step = 0.01, description = 'μ (mean)'),
         sigma = FloatSlider(value = 0.01, min = 0.01, max = 10.0, step = 0.01, description = 'σ (std)'))

interactive(children=(FloatSlider(value=0.0, description='μ (mean)', max=10.0, min=-10.0, step=0.01), FloatSli…

<function __main__.plot_combined(mu=0.0, sigma=1.0)>

In [36]:
xx = np.linspace(-10, 10, 301)
fxx = [np.exp(bs_model3.log_density(np.array([xxn]), propto = False)) for xxn in xx]

interact(plot_combined,
         mu = FloatSlider(value = 0.0, min = -10.0, max = 10.0, step = 0.01, description = 'μ (mean)'),
         sigma = FloatSlider(value = 0.01, min = 0.01, max = 10.0, step = 0.01, description = 'σ (std)'))

interactive(children=(FloatSlider(value=0.0, description='μ (mean)', max=10.0, min=-10.0, step=0.01), FloatSli…

<function __main__.plot_combined(mu=0.0, sigma=1.0)>